In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from keras.datasets import mnist

tf.logging.set_verbosity(tf.logging.INFO)

Using TensorFlow backend.


In [2]:
tf.__version__

'1.11.0'

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
print("Train Set: {}".format(x_train.shape))
print("Test Set: {}".format(x_test.shape))

Train Set: (60000, 28, 28)
Test Set: (10000, 28, 28)


In [5]:
print("Train Set: {}".format(y_train.shape))
print("Test Set: {}".format(y_test.shape))

Train Set: (60000,)
Test Set: (10000,)


In [6]:
x_train = x_train/255.0
x_test = x_test/255.0

In [7]:
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

In [8]:
print("Train Set: {}".format(y_train.shape))
print("Test Set: {}".format(y_test.shape))

Train Set: (60000, 10)
Test Set: (10000, 10)


In [9]:
def cnn_model_fn(features, labels, mode):
    
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
                              inputs=input_layer,
                              filters=32,
                              kernel_size=[5, 5],
                              padding="same",
                              activation=tf.nn.relu
                            )

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(
                                    inputs=conv1, 
                                    pool_size=[2, 2], 
                                    strides=2
                                    )

    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
                              inputs=pool1,
                              filters=64,
                              kernel_size=[5, 5],
                              padding="same",
                              activation=tf.nn.relu
                            )
    
    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(
                                    inputs=conv2, 
                                    pool_size=[2, 2], 
                                    strides=2
                                    )

    # Dense Layer
    pool2_flat = tf.reshape(
                            tensor=pool2, 
                            shape=[-1, 7 * 7 * 64]
                            )
    
    dense = tf.layers.dense(
                            inputs=pool2_flat, 
                            units=1024, 
                            activation=tf.nn.relu
                            )
    
    dropout = tf.layers.dropout(
                                inputs=dense, 
                                rate=0.4, 
                                training=mode == tf.estimator.ModeKeys.TRAIN
                                )

    # Logits Layer
    logits = tf.layers.dense(
                            inputs=dropout, 
                            units=10
                            )

    # Generate predictions (for PREDICT and EVAL mode)
    predictions = {
                      "classes": tf.argmax(input=logits, axis=1),
                      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
                    }

    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=tf.argmax(labels, axis=1), logits=logits)

    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
                                        loss=loss,
                                        global_step=tf.train.get_global_step()
                                      )
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    
    # Add evaluation metrics (for EVAL mode)
    accuracy= tf.metrics.accuracy(labels=labels,predictions=predictions["classes"]) )
                        }
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [10]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdccdd38828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [12]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x = {"x": x_train},
                    y = y_train.astype(np.int32),
                    batch_size=64,
                    num_epochs = None,
                    shuffle = True
)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"x": x_test},
                    y=y_test.astype(np.int32),
                    num_epochs=1,
                    shuffle=False
                )

In [ ]:
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=2000,
    hooks=[logging_hook]
)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.09229369 0.10859552 0.09324619 0.11060697 0.10319494 0.09608488
  0.09171301 0.0894925  0.1082633  0.106509  ]
 [0.11165351 0.12136837 0.09288162 0.10195451 0.10361398 0.09347236
  0.08394179 0.09390916 0.11029015 0.08691456]
 [0.0980396  0.10844562 0.10393045 0.09129673 0.09681458 0.10545353
  0.08595615 0.09665136 0.11299901 0.10041296]
 [0.09591043 0.10684056 0.10999549 0.09712595 0.093685

INFO:tensorflow:loss = 2.297877311706543, step = 0
INFO:tensorflow:probabilities = [[0.10594196 0.10165226 0.10984024 0.0942014  0.09566543 0.09577549
  0.09282948 0.09964018 0.1072861  0.09716745]
 [0.09016481 0.09364558 0.09912598 0.10639158 0.09589284 0.09202342
  0.09869905 0.10885088 0.12183643 0.09336944]
 [0.08807863 0.10983522 0.1155128  0.09359654 0.09363884 0.09873344
  0.10966518 0.08406704 0.11198004 0.09489229]
 [0.10748459 0.09740014 0.0880371  0.10574322 0.10103479 0.09476074
  0.09087522 0.10388452 0.1149389  0.09584076]
 [0.10348015 0.10615553 0.10159284 0.10530412 0.10250296 0.09594118
  0.09198226 0.09471158 0.10691914 0.09141023]
 [0.09366925 0.1112927  0.10115495 0.10578576 0.09925872 0.09294446
  0.08343923 0.10179605 0.11476232 0.09589655]
 [0.11123568 0.1179216  0.09524896 0.09376158 0.09715118 0.08954637
  0.0920753  0.11603305 0.0948112  0.09221508]
 [0.10499383 0.09835088 0.08983414 0.10692896 0.09727143 0.08326566
  0.10637085 0.08700709 0.12522726 0.1007499

INFO:tensorflow:loss = 2.2948191165924072, step = 100 (5.310 sec)
INFO:tensorflow:probabilities = [[0.10098908 0.09587045 0.08832651 0.09508755 0.10846964 0.09305015
  0.08813084 0.12813245 0.10974798 0.09219535]
 [0.10945285 0.10410564 0.09493377 0.09747693 0.11097244 0.08859561
  0.08700309 0.09935728 0.1027325  0.10536989]
 [0.09674038 0.11883169 0.10045563 0.10177847 0.10214198 0.09524553
  0.08821513 0.09097289 0.10947695 0.09614136]
 [0.10230747 0.10566103 0.10690574 0.10034047 0.10116692 0.08905526
  0.08841382 0.09285654 0.10909041 0.10420233]
 [0.11121314 0.09283603 0.10034198 0.10812208 0.0951227  0.09446962
  0.08882051 0.09749783 0.11523938 0.09633673]
 [0.10790223 0.10453615 0.09573911 0.09352972 0.09632989 0.08463728
  0.10406613 0.09853618 0.12291298 0.09181033]
 [0.11018117 0.09966954 0.10580931 0.09978138 0.09835622 0.0880658
  0.091475   0.10140371 0.10687815 0.0983797 ]
 [0.11331224 0.10464786 0.10716596 0.11168778 0.09139977 0.07713128
  0.09227957 0.09507127 0.1055

INFO:tensorflow:loss = 2.2566604614257812, step = 200 (5.127 sec)
INFO:tensorflow:probabilities = [[0.1015494  0.10580285 0.08681485 0.09194305 0.10111432 0.08082324
  0.09773376 0.11352376 0.12553036 0.09516442]
 [0.0924584  0.0902693  0.10204965 0.12023255 0.09484875 0.09093587
  0.08128716 0.11158442 0.12326307 0.09307083]
 [0.1059496  0.11501124 0.09033045 0.1006713  0.1181651  0.07970513
  0.1049083  0.09203718 0.1010885  0.09213319]
 [0.09885025 0.10150119 0.08987114 0.11064712 0.08764408 0.09693376
  0.09776934 0.09901605 0.11898857 0.0987785 ]
 [0.10074103 0.10691252 0.09799559 0.10459587 0.09515495 0.09590481
  0.09245841 0.09696473 0.11410382 0.09516827]
 [0.12525063 0.07295084 0.0983524  0.11070287 0.10487259 0.08417051
  0.10408656 0.10485072 0.12399633 0.07076653]
 [0.12150808 0.09038958 0.11259943 0.10751901 0.08775307 0.09285862
  0.08119124 0.09299657 0.12042468 0.09275971]
 [0.11239433 0.09990309 0.11316345 0.10165241 0.10376078 0.08351358
  0.08859649 0.11296324 0.093

INFO:tensorflow:loss = 2.2189760208129883, step = 300 (5.296 sec)
INFO:tensorflow:probabilities = [[0.0960395  0.08498671 0.11334293 0.11642161 0.10015675 0.09081163
  0.09651209 0.08419141 0.12772069 0.08981668]
 [0.10627485 0.11182483 0.08830845 0.0938306  0.10313155 0.1032165
  0.09562903 0.08448256 0.11897885 0.09432279]
 [0.10607149 0.10090954 0.11140364 0.11221199 0.09504201 0.09360601
  0.09206551 0.09497241 0.10838547 0.08533193]
 [0.10057522 0.11355875 0.11376063 0.10027567 0.09245942 0.09821837
  0.08870576 0.09150027 0.11166282 0.08928308]
 [0.11513701 0.09712958 0.10803589 0.09864169 0.0977843  0.0938415
  0.08417996 0.09567556 0.10658843 0.10298608]
 [0.10852266 0.10554794 0.10850899 0.09944252 0.11985303 0.07551329
  0.08798986 0.09144191 0.11278641 0.0903934 ]
 [0.09393318 0.1108202  0.0987277  0.11018013 0.09622345 0.09260183
  0.09856505 0.10047282 0.10980986 0.0886658 ]
 [0.11041813 0.09627287 0.1041648  0.09251921 0.11194552 0.08444398
  0.09582273 0.09648914 0.10474

In [ ]:
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)